In [31]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

## Weather

In [32]:
weather = pd.read_csv('/Users/annaywj/Desktop/SDSU/BDA600/Capstone/sd_weather_2018_2024_combined.csv')
crash = pd.read_csv('/Users/annaywj/Desktop/SDSU/BDA600/Capstone/TIMS_SD_Crashes2013-2024.csv')

In [33]:
weather['datetime'] = pd.to_datetime(weather['datetime'], errors='coerce')
crash['COLLISION_DATE'] = pd.to_datetime(crash['COLLISION_DATE'], errors='coerce')

# Filter crash data for 2018–2024
crash_filtered = crash[(crash['COLLISION_DATE'].dt.year >= 2018) &
                          (crash['COLLISION_DATE'].dt.year <= 2024)].copy()

# Aggregate crash data by date
daily_crashes = crash_filtered.groupby('COLLISION_DATE').agg(
    TOTAL_CRASHES=('CASE_ID', 'count'),
    AVG_SEVERITY=('COLLISION_SEVERITY', 'mean')
).reset_index().rename(columns={'COLLISION_DATE': 'datetime'})

# Merge with weather data
merged_weather = pd.merge(weather, daily_crashes, on='datetime', how='inner')

# Select weather predictors and target
weather_features = ['humidity', 'cloudcover', 'windspeed', 'precip']
target_column = 'AVG_SEVERITY'

# Drop missing values
regression_ready = merged_weather[weather_features + [target_column]].dropna()

In [34]:
X = regression_ready[weather_features]
y = regression_ready[target_column]

# Add constant for OLS regression
X_ols = sm.add_constant(X)
ols_model = sm.OLS(y, X_ols).fit()

# Random Forest Regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rf_r2 = r2_score(y_test, y_pred)
rf_importance = pd.Series(rf_model.feature_importances_, index=weather_features).sort_values(ascending=False)

# Output results
ols_summary = ols_model.summary()
rf_r2, rf_importance, ols_summary.tables[1]

(-0.10411692103061565,
 cloudcover    0.359832
 humidity      0.321860
 windspeed     0.269387
 precip        0.048921
 dtype: float64,
 <class 'statsmodels.iolib.table.SimpleTable'>)

## SOC

In [35]:
road = pd.read_csv('/Users/annaywj/Downloads/SOC_-_Local_Roads__Speed_and_Volume_20250423.csv')

In [36]:

crash['COLLISION_DATE'] = pd.to_datetime(crash['COLLISION_DATE'], errors='coerce')
crash_filtered = crash[(crash['COLLISION_DATE'].dt.year >= 2018) & 
                       (crash['COLLISION_DATE'].dt.year <= 2024)].copy()

# Drop rows without location
crash_filtered = crash_filtered.dropna(subset=['POINT_X', 'POINT_Y'])

# Convert to GeoDataFrame
crash_gdf = gpd.GeoDataFrame(
    crash_filtered,
    geometry=gpd.points_from_xy(crash_filtered['POINT_X'], crash_filtered['POINT_Y']),
    crs="EPSG:4326"
)

# Step 2: Convert road data to GeoDataFrame
road = road.dropna(subset=['geometry'])
road_gdf = gpd.GeoDataFrame(road, geometry=gpd.GeoSeries.from_wkt(road['geometry']), crs="EPSG:4326")

# Step 3: Project both to meters
crash_gdf = crash_gdf.to_crs(epsg=3857)
road_gdf = road_gdf.to_crs(epsg=3857)

# Step 4: Use midpoint of road segment
road_gdf['rep_point'] = road_gdf.geometry.representative_point()
road_gdf.set_geometry('rep_point', inplace=True)

# Step 5: Nearest neighbor match
crash_coords = np.array(list(zip(crash_gdf.geometry.x, crash_gdf.geometry.y)))
road_coords = np.array(list(zip(road_gdf.geometry.x, road_gdf.geometry.y)))

nn = NearestNeighbors(n_neighbors=1, radius=50)
nn.fit(road_coords)
distances, indices = nn.kneighbors(crash_coords)

within_50m = distances[:, 0] <= 50
crash_gdf = crash_gdf[within_50m]
matched_indices = indices[within_50m].flatten()

# Step 6: Merge
matched_roads_clean = road_gdf.reset_index().iloc[matched_indices].reset_index(drop=True).drop(columns=['geometry'])
joined_df = pd.concat([crash_gdf.reset_index(drop=True), matched_roads_clean], axis=1)

# Step 7: Aggregate
aggregated = joined_df.groupby('osm_id').agg(
    TOTAL_CRASHES=('CASE_ID', 'count'),
    AVG_SEVERITY=('COLLISION_SEVERITY', 'mean'),
    Lanes=('Lanes', 'first'),
    Speed_Limit_MPH=('Speed Limit MPH', 'first'),
    Speed_2022_MPH=('Speed 2022 MPH', 'first'),
    Speed_Change=('1 year Speed % change', 'first'),
    AADT_Change=('1 year AADT % change', 'first')
).dropna()

# Step 8: Regression
X = aggregated[['Lanes', 'Speed_Limit_MPH', 'Speed_2022_MPH', 'Speed_Change', 'AADT_Change']]
y = aggregated['AVG_SEVERITY']

X_ols = sm.add_constant(X)
ols_model = sm.OLS(y, X_ols).fit()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rf_r2 = r2_score(y_test, y_pred)
rf_importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

# Output
ols_summary = ols_model.summary()
rf_r2, rf_importance, ols_summary.tables[1]

(-0.10429588222092101,
 AADT_Change        0.309248
 Speed_2022_MPH     0.270044
 Speed_Change       0.259344
 Lanes              0.089369
 Speed_Limit_MPH    0.071995
 dtype: float64,
 <class 'statsmodels.iolib.table.SimpleTable'>)

## Ped_party & Ped_Victim

In [37]:
ped_parties = pd.read_csv('/Users/annaywj/Downloads/Ped_Parties.csv')
ped_victims = pd.read_csv('/Users/annaywj/Downloads/Ped_Victims.csv')

In [38]:
socio_party_merged = ped_parties.merge(
    crash[['CASE_ID', 'COLLISION_SEVERITY']],
    on='CASE_ID',
    how='left'
)

# Map AT_FAULT to binary
socio_party_merged['AT_FAULT_BINARY'] = socio_party_merged['AT_FAULT'].map({'Y': 1, 'N': 0})

# Step 2: Merge in victim data (victim-level table with PARTY + CRASH info)
socio_party_victim_merged = socio_party_merged.merge(
    ped_victims[['CASE_ID', 'PARTY_NUMBER', 'VICTIM_AGE', 'VICTIM_DEGREE_OF_INJURY']],
    on=['CASE_ID', 'PARTY_NUMBER'],
    how='left'
)

# Step 3: One-hot encode victim injury severity
socio_party_victim_merged = pd.get_dummies(
    socio_party_victim_merged,
    columns=['VICTIM_DEGREE_OF_INJURY'],
    prefix='INJURY',
    drop_first=True
)

# Step 4: Prepare feature set
model_features_victims = ['PARTY_AGE', 'VICTIM_AGE', 'AT_FAULT_BINARY'] + \
    [col for col in socio_party_victim_merged.columns if col.startswith('INJURY_')]

model_data_victims = socio_party_victim_merged[model_features_victims + ['COLLISION_SEVERITY']].dropna()

X = model_data_victims[model_features_victims]
y = model_data_victims['COLLISION_SEVERITY']

# Step 5: Run OLS
X_ols = sm.add_constant(X)
ols_model = sm.OLS(y, X_ols).fit()

# Step 6: Random Forest Regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
rf_r2 = r2_score(y_test, y_pred)
rf_importance = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)

# Output model results
ols_summary = ols_model.summary()
rf_r2, rf_importance, ols_summary.tables[1]

(0.6957053396275799,
 INJURY_4.0         0.436046
 INJURY_1.0         0.228706
 INJURY_2.0         0.094561
 PARTY_AGE          0.090444
 VICTIM_AGE         0.089447
 INJURY_5.0         0.030886
 INJURY_7.0         0.014858
 AT_FAULT_BINARY    0.006772
 INJURY_3.0         0.004173
 INJURY_6.0         0.004107
 dtype: float64,
 <class 'statsmodels.iolib.table.SimpleTable'>)

## Final Model

In [39]:
weather_vars = ['cloudcover', 'humidity', 'windspeed']
weather_final = merged_weather[['datetime', 'AVG_SEVERITY'] + weather_vars].dropna()

# From road: aggregated per road segment with osm_id
road_vars = ['AADT_Change', 'Speed_2022_MPH', 'Speed_Change']
road_final = aggregated[road_vars + ['AVG_SEVERITY']].dropna()

# From party/victim: victim-level merged with CRASH info
party_victim_vars = ['PARTY_AGE', 'VICTIM_AGE', 'INJURY_1.0', 'INJURY_2.0', 'INJURY_4.0']
victim_final = socio_party_victim_merged[party_victim_vars + ['COLLISION_SEVERITY']].dropna()

# Rename severity columns to align
weather_final = weather_final.rename(columns={'AVG_SEVERITY': 'severity'})
road_final = road_final.rename(columns={'AVG_SEVERITY': 'severity'})
victim_final = victim_final.rename(columns={'COLLISION_SEVERITY': 'severity'})

# Step 2: Create separate models for each (different levels — can't merge directly)
X_weather = weather_final[weather_vars]
X_road = road_final[road_vars]
X_victim = victim_final[party_victim_vars]
y_weather = weather_final['severity']
y_road = road_final['severity']
y_victim = victim_final['severity']

# Step 3: Train Random Forest models on each
def train_rf_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
    return r2, importance

r2_weather, imp_weather = train_rf_model(X_weather, y_weather)
r2_road, imp_road = train_rf_model(X_road, y_road)
r2_victim, imp_victim = train_rf_model(X_victim, y_victim)

r2_weather, imp_weather, r2_road, imp_road, r2_victim, imp_victim

(-0.11099561591030516,
 cloudcover    0.376267
 humidity      0.338181
 windspeed     0.285551
 dtype: float64,
 -0.1329219627770124,
 AADT_Change       0.382801
 Speed_Change      0.312471
 Speed_2022_MPH    0.304728
 dtype: float64,
 0.648014207470236,
 INJURY_4.0    0.480172
 INJURY_1.0    0.251844
 INJURY_2.0    0.104129
 PARTY_AGE     0.084259
 VICTIM_AGE    0.079596
 dtype: float64)

In [40]:
road_vars_final = ['1 year AADT % change', '1 year Speed % change', 'Speed 2022 MPH']

In [41]:
final_features_corrected = [
    'COLLISION_SEVERITY',
    'cloudcover', 'humidity', 'windspeed',
    '1 year AADT % change', '1 year Speed % change', 'Speed 2022 MPH',
    'PARTY_AGE', 'VICTIM_AGE',
    'INJURY_1.0', 'INJURY_2.0', 'INJURY_4.0'
]

# Subset and drop rows with missing data
final_model_data = final_merged[final_features_corrected].dropna()

# Define X and y
X_final = final_model_data.drop(columns=['COLLISION_SEVERITY'])
y_final = final_model_data['COLLISION_SEVERITY']

# Train final random forest model
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)
final_rf = RandomForestRegressor(random_state=42)
final_rf.fit(X_train, y_train)
y_pred = final_rf.predict(X_test)
final_r2 = r2_score(y_test, y_pred)
final_importance = pd.Series(final_rf.feature_importances_, index=X_final.columns).sort_values(ascending=False)

final_r2, final_importance

(0.7571386668620821,
 INJURY_4.0               0.394914
 INJURY_1.0               0.201598
 INJURY_2.0               0.093922
 cloudcover               0.046850
 Speed 2022 MPH           0.045934
 humidity                 0.044600
 windspeed                0.040745
 1 year AADT % change     0.037626
 1 year Speed % change    0.037290
 VICTIM_AGE               0.029706
 PARTY_AGE                0.026814
 dtype: float64)

In [42]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Store models and their names
models = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "Ridge": Ridge(random_state=42),
    "Lasso": Lasso(random_state=42),
    "Polynomial RF (2nd degree)": make_pipeline(
        PolynomialFeatures(degree=2, include_bias=False), 
        RandomForestRegressor(random_state=42)
    )
}

# Train and evaluate each model
model_r2_results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    model_r2_results[name] = r2_score(y_test, preds)

model_r2_results_sorted = dict(sorted(model_r2_results.items(), key=lambda item: item[1], reverse=True))
model_r2_results_sorted


{'Random Forest': 0.7571386668620821,
 'Polynomial RF (2nd degree)': 0.7484760765122647,
 'XGBoost': 0.7224903992631877,
 'Gradient Boosting': 0.7068791645230569,
 'Ridge': 0.6925502649149384,
 'Lasso': -0.0007030623447410456}

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor

# 1. Hyperparameter Tuning for Random Forest
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [10, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt"]
}

rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3, n_jobs=-1)
rf_grid.fit(X_train, y_train)
rf_grid_best = rf_grid.best_estimator_
rf_grid_r2 = r2_score(y_test, rf_grid_best.predict(X_test))

# 2. Feature Enrichment (add interaction term: speed × cloudcover)
X_enriched = X_final.copy()
X_enriched['speed_cloud_interaction'] = X_enriched['Speed 2022 MPH'] * X_enriched['cloudcover']
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_enriched, y_final, test_size=0.3, random_state=42)
rf_enriched = RandomForestRegressor(random_state=42)
rf_enriched.fit(X_train_e, y_train_e)
rf_enriched_r2 = r2_score(y_test_e, rf_enriched.predict(X_test_e))

# 3. Sample Weighting by Severity Level
weights = y_train.map({1: 3, 2: 2, 3: 1, 4: 1})  # More weight for fatal and severe
rf_weighted = RandomForestRegressor(random_state=42)
rf_weighted.fit(X_train, y_train, sample_weight=weights)
rf_weighted_r2 = r2_score(y_test, rf_weighted.predict(X_test))

# 4. Ensemble Stacking
stack_model = StackingRegressor(
    estimators=[
        ("rf", RandomForestRegressor(random_state=42)),
        ("gb", GradientBoostingRegressor(random_state=42)),
        ("ridge", Ridge(random_state=42))
    ],
    final_estimator=Ridge()
)
stack_model.fit(X_train, y_train)
stack_r2 = r2_score(y_test, stack_model.predict(X_test))

# Return all updated R² scores
improved_model_r2 = {
    "Tuned RF": rf_grid_r2,
    "RF + Interaction Term": rf_enriched_r2,
    "RF with Weights": rf_weighted_r2,
    "Stacked Ensemble": stack_r2
}
improved_model_r2


{'Tuned RF': 0.7528748562311883,
 'RF + Interaction Term': 0.7606886345113067,
 'RF with Weights': 0.7542038702072595,
 'Stacked Ensemble': 0.7589460085698958}